In [2]:
import csv
from io import StringIO
from urllib.request import urlopen

In [5]:
STUDENT_CSV = "startup-ideas-2018.csv"
ALL_STUDENTS = []
SKIP_SECTION_MIX = False
SKIP_MBA_MIX = False
SECTIONS = ['Section 1', 'Section 2', 'Section 3']

csv_reader = csv.reader(open(STUDENT_CSV))
next(csv_reader)
for student in csv_reader:
    (last_name, first_name, email, program, section) = student
    ALL_STUDENTS.append({ 'section': section, 'id': email, 'first_name': first_name, 'last_name': last_name, 'program': program })

print("Total number of students: %d." % len(ALL_STUDENTS))
for section in SECTIONS:
    students_in_section = [s for s in ALL_STUDENTS if s['section'] == section]
    print("%d student(s) in %s." % (len(students_in_section), section))

Total number of students: 253.
85 student(s) in Section 1.
70 student(s) in Section 2.
98 student(s) in Section 3.


In [7]:
ALL_STUDENTS = [s for s in ALL_STUDENTS if s['section'] == 'Section 1']

In [8]:
import math
import random

MIN_TEAM_SIZE = 6
MAX_TEAM_SIZE = MIN_TEAM_SIZE + 1
NUMBER_OF_TEAMS = math.floor(len(ALL_STUDENTS) / MIN_TEAM_SIZE)
RANDOM = True

ALL_TEAMS = [ 'Team%03d' % k for k in range (1, NUMBER_OF_TEAMS+1)]

if RANDOM:
    random.shuffle(ALL_STUDENTS)

In [23]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver('StudentProjectGridCBC', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

matches = {}
for student in ALL_STUDENTS:
    for team in ALL_TEAMS:
        matches[student['id'], team] = solver.IntVar(0, 1, 'matches[%s,%s]' % (student['id'], team))

z = solver.Sum(  1 * matches[student['id'], team]
                 for student in ALL_STUDENTS
                 for team in ALL_TEAMS)

for student in ALL_STUDENTS:
    solver.Add(solver.Sum([matches[student['id'], team] for team in ALL_TEAMS]) == 1)
print("`One team per student` added.")

# Between MIN_TEAM_SIZE and MAX_TEAM_SIZE students per project
for team in ALL_TEAMS:
    solver.Add(solver.Sum([matches[student['id'], team] for student in ALL_STUDENTS]) >= MIN_TEAM_SIZE)
    solver.Add(solver.Sum([matches[student['id'], team] for student in ALL_STUDENTS]) <= MAX_TEAM_SIZE)
print("`Team size added.`")


#  1 < MBA <= 2
for team in ALL_TEAMS:
    solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'MBA' else 0) for student in ALL_STUDENTS]) <= 2)
    solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'MBA' else 0) for student in ALL_STUDENTS]) >= 1)

#  LLM <= 1
for team in ALL_TEAMS:
    solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'LLM' else 0) for student in ALL_STUDENTS]) <= 1)

#  CS <= 2
for team in ALL_TEAMS:
    solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'CS' else 0) for student in ALL_STUDENTS]) <= 2)

# ORIE <= 2
for team in ALL_TEAMS:
    solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'ORIE' else 0) for student in ALL_STUDENTS]) <= 2)

# Jacobs <= 2
for team in ALL_TEAMS:
    solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] in ['CM', 'HT'] else 0) for student in ALL_STUDENTS]) <= 2)

    
# At least one technical student
#for team in ALL_TEAMS:
#    solver.Add(solver.Sum([matches[student['id'], team] * (1 if student['program'] == 'Master of Engineering Programs' else 0) for student in ALL_STUDENTS]) >= 1)


objective = solver.Minimize(z)
solver.Solve()
s = solver.Objective().Value()
print(s)

`One team per student` added.
`Team size added.`
85.0


In [24]:
BUCKETS = {}
for team in ALL_TEAMS:
    BUCKETS[team] = []

for student in ALL_STUDENTS:
    for team in ALL_TEAMS:
        if int(matches[student['id'], team].SolutionValue()):
            BUCKETS[team].append(student)

for index, team in enumerate(ALL_TEAMS):
    print("%s (%d people)" % (team, len(BUCKETS[team])))
    print('-' * 20)
    for student in BUCKETS[team]:
        email = student['id'] if '@' in student['id'] else "%s@cornell.edu" % student['id']
        print('"%s %s",%s,%s,%s,%s' % (student['first_name'], student['last_name'], email, team, student['section'], student['program']))
    print()

Team001 (6 people)
--------------------
"Kibum Byun",kb656@cornell.edu,Team001,Section 1,LLM
"Gabriel Soares Pena Costa",gs669@cornell.edu,Team001,Section 1,MBA
"Xinzhuo Sun",xs324@cornell.edu,Team001,Section 1,CS
"Anthony Bisulco",arb426@cornell.edu,Team001,Section 1,ECE
"Haojie Zhang",hz538@cornell.edu,Team001,Section 1,ORIE
"Chenyu Zhang",cz442@cornell.edu,Team001,Section 1,CS

Team002 (6 people)
--------------------
"Yuchen Shi",ys838@cornell.edu,Team002,Section 1,CM
"Jillian Burns",jjb427@cornell.edu,Team002,Section 1,CS
"Natalie Chyi",nhc36@cornell.edu,Team002,Section 1,LLM
"Alana Lipson",ajl336@cornell.edu,Team002,Section 1,MBA
"Rahul Naryani",rsn53@cornell.edu,Team002,Section 1,MBA
"Kulvinder Lotay",ksl76@cornell.edu,Team002,Section 1,CS

Team003 (6 people)
--------------------
"Young Sang Choi",yc2376@cornell.edu,Team003,Section 1,HT
"Yaohai Xu",yx432@cornell.edu,Team003,Section 1,CS
"Pooja Kale",pk576@cornell.edu,Team003,Section 1,CS
"Saniya Shah",ss3734@cornell.edu,Team003,S